# Creating `movie-tiles` with Earth Engine

## [Tiled web map](https://en.wikipedia.org/wiki/Tiled_web_map)

Most tiled web maps follow certain Google Maps conventions:

- Tiles are 256x256 pixels
- At the outer most zoom level, 0, the entire world can be rendered in a single map tile.
- Each zoom level doubles in both dimensions, so a single tile is replaced by 4 tiles when zooming in. 
- The [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator_projection) projection is used, with latitude limits of around 85 degrees.

OpenStreetMap, Google Maps, MapBox, etc. use the standard known as [Slippy Map Tilenames](https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames) or XYZ:

- Tiles are 256 × 256 pixel PNG files
- Each zoom level (z) is a directory, each column (x) is a subdirectory, and each tile (y) in that column is a file
- Filename (url) format is `http://.../z/x/y.png`

Other standards also exist such as the [Tile Map Service](https://en.wikipedia.org/wiki/Tile_Map_Service) or TMS.

The only difference between XYZ and TMS is a flip in coordinates. TMS has its origin [0,0] at the bottom-left, while the origin for XYZ is located at the top-left.

The conversion between tiles and coordinates has been described [here](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/) ([source code](https://gist.github.com/maptiler/fddb5ce33ba995d5523de9afdf8ef118#file-globalmaptiles-py)). 

In [1]:
import math

class GlobalMercator(object):
    """
    TMS Global Mercator Profile
    ---------------------------
    Functions necessary for generation of tiles in Spherical Mercator projection,
    EPSG:900913 (EPSG:gOOglE, Google Maps Global Mercator), EPSG:3785, OSGEO:41001.
    Such tiles are compatible with Google Maps, Microsoft Virtual Earth, Yahoo Maps,
    UK Ordnance Survey OpenSpace API, ...
    and you can overlay them on top of base maps of those web mapping applications.
    
    Pixel and tile coordinates are in TMS notation (origin [0,0] in bottom-left).
    What coordinate conversions do we need for TMS Global Mercator tiles::
         LatLon      <->       Meters      <->     Pixels    <->       Tile     
     WGS84 coordinates   Spherical Mercator  Pixels in pyramid  Tiles in pyramid
         lat/lon            XY in metres     XY pixels Z zoom      XYZ from TMS 
        EPSG:4326           EPSG:900913                                         
         .----.              ---------               --                TMS      
        /      \     <->     |       |     <->     /----/    <->      Google    
        \      /             |       |           /--------/          QuadTree   
         -----               ---------         /------------/                   
       KML, public         WebMapService         Web Clients      TileMapService
    What is the coordinate extent of Earth in EPSG:900913?
      [-20037508.342789244, -20037508.342789244, 20037508.342789244, 20037508.342789244]
      Constant 20037508.342789244 comes from the circumference of the Earth in meters,
      which is 40 thousand kilometers, the coordinate origin is in the middle of extent.
      In fact you can calculate the constant as: 2 * math.pi * 6378137 / 2.0
      $ echo 180 85 | gdaltransform -s_srs EPSG:4326 -t_srs EPSG:900913
      Polar areas with abs(latitude) bigger then 85.05112878 are clipped off.
    What are zoom level constants (pixels/meter) for pyramid with EPSG:900913?
      whole region is on top of pyramid (zoom=0) covered by 256x256 pixels tile,
      every lower zoom level resolution is always divided by two
      initialResolution = 20037508.342789244 * 2 / 256 = 156543.03392804062
    What is the difference between TMS and Google Maps/QuadTree tile name convention?
      The tile raster itself is the same (equal extent, projection, pixel size),
      there is just different identification of the same raster tile.
      Tiles in TMS are counted from [0,0] in the bottom-left corner, id is XYZ.
      Google placed the origin [0,0] to the top-left corner, reference is XYZ.
      Microsoft is referencing tiles by a QuadTree name, defined on the website:
      http://msdn2.microsoft.com/en-us/library/bb259689.aspx
    The lat/lon coordinates are using WGS84 datum, yeh?
      Yes, all lat/lon we are mentioning should use WGS84 Geodetic Datum.
      Well, the web clients like Google Maps are projecting those coordinates by
      Spherical Mercator, so in fact lat/lon coordinates on sphere are treated as if
      the were on the WGS84 ellipsoid.
     
      From MSDN documentation:
      To simplify the calculations, we use the spherical form of projection, not
      the ellipsoidal form. Since the projection is used only for map display,
      and not for displaying numeric coordinates, we don't need the extra precision
      of an ellipsoidal projection. The spherical projection causes approximately
      0.33 percent scale distortion in the Y direction, which is not visually noticable.
    How do I create a raster in EPSG:900913 and convert coordinates with PROJ.4?
      You can use standard GIS tools like gdalwarp, cs2cs or gdaltransform.
      All of the tools supports -t_srs 'epsg:900913'.
      For other GIS programs check the exact definition of the projection:
      More info at http://spatialreference.org/ref/user/google-projection/
      The same projection is degined as EPSG:3785. WKT definition is in the official
      EPSG database.
      Proj4 Text:
        +proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0
        +k=1.0 +units=m +nadgrids=@null +no_defs
      Human readable WKT format of EPGS:900913:
         PROJCS["Google Maps Global Mercator",
             GEOGCS["WGS 84",
                 DATUM["WGS_1984",
                     SPHEROID["WGS 84",6378137,298.2572235630016,
                         AUTHORITY["EPSG","7030"]],
                     AUTHORITY["EPSG","6326"]],
                 PRIMEM["Greenwich",0],
                 UNIT["degree",0.0174532925199433],
                 AUTHORITY["EPSG","4326"]],
             PROJECTION["Mercator_1SP"],
             PARAMETER["central_meridian",0],
             PARAMETER["scale_factor",1],
             PARAMETER["false_easting",0],
             PARAMETER["false_northing",0],
             UNIT["metre",1,
                 AUTHORITY["EPSG","9001"]]]
    """
    
    def __init__(self, tileSize=256):
        "Initialize the TMS Global Mercator pyramid"
        self.tileSize = tileSize
        self.initialResolution = 2 * math.pi * 6378137 / self.tileSize
        # 156543.03392804062 for tileSize 256 pixels
        self.originShift = 2 * math.pi * 6378137 / 2.0
        # 20037508.342789244
        
    def MetersToLatLon(self, mx, my ):
        "Converts XY point from Spherical Mercator EPSG:900913 to lat/lon in WGS84 Datum"

        lon = (mx / self.originShift) * 180.0
        lat = (my / self.originShift) * 180.0

        lat = 180 / math.pi * (2 * math.atan( math.exp( lat * math.pi / 180.0)) - math.pi / 2.0)
        return lat, lon

    def Resolution(self, zoom ):
        "Resolution (meters/pixel) for given zoom level (measured at Equator)"
        
        # return (2 * math.pi * 6378137) / (self.tileSize * 2**zoom)
        return self.initialResolution / (2**zoom)
        
    def PixelsToMeters(self, px, py, zoom):
        "Converts pixel coordinates in given zoom level of pyramid to EPSG:900913"

        res = self.Resolution( zoom )
        mx = px * res - self.originShift
        my = py * res - self.originShift
        return mx, my
        

    def TileBounds(self, tx, ty, zoom):
        "Returns bounds of the given tile in EPSG:900913 coordinates"
        
        minx, miny = self.PixelsToMeters( tx*self.tileSize, ty*self.tileSize, zoom )
        maxx, maxy = self.PixelsToMeters( (tx+1)*self.tileSize, (ty+1)*self.tileSize, zoom )
        return ( minx, miny, maxx, maxy )
        
    def TileLonLatBounds(self, tx, ty, zoom ):
        "Returns bounds of the given tile in latutude/longitude using WGS84 datum"

        bounds = self.TileBounds( tx, ty, zoom)
        minLat, minLon = self.MetersToLatLon(bounds[0], bounds[1])
        maxLat, maxLon = self.MetersToLatLon(bounds[2], bounds[3])
         
        return ( minLon, minLat, maxLon, maxLat )
    
    def GoogleTile(self, tx, ty, zoom):
        "Converts TMS tile coordinates to Google Tile coordinates and vice versa"
        
        # coordinate origin is moved from bottom-left to top-left corner of the extent
        return tx, (2**zoom - 1) - ty

## Exporting the `movie-tiles` to a Cloud Storage bucket

In [2]:
import numpy as np
import math
import ee

GM = GlobalMercator()

class ee_movie_tiles:
    
    def __init__(self, imageCollection, startDate, stopDate, minZoom, maxZoom, visParam, bucket_name, folder_path):
        """
        Class used to get the datasets from Earth Engine
        Parameters
        ----------
        imageCollection: string
            Name of the Image Collection.
        startDate : string
            Starting data
        stopDate : string
            Stopping data
        minZoom : integer
            The minimum zoom level of the map tiles to export.
        maxZoom : integer
            The maximum zoom level of the map tiles to export.
        visParam: dictionary
            Image visualization parameters.
        bucket_name: string
            Google Cloud Bucket
        folder_path: string
            Folder path in the bucke.
        """
 
        self.image_collection = imageCollection
        self.startDate = startDate
        self.stopDate = stopDate 
        self.minZoom = minZoom
        self.maxZoom = maxZoom
        self.vis = visParam
        self.bucket = bucket_name
        self.path = folder_path
        
    def TileCoordinates (self,z):
        """Tile coordinates for a given z"""
        return np.arange(int(np.sqrt(4**z)))
    
    def vis_param(self,image):
        """Map visualization parameters"""
        return ee.Image(image.visualize(**self.vis))
    
    def export_movie(self):
        
        ## Define your collection
        self.collection = ee.ImageCollection(self.image_collection).filterDate(self.startDate,self.stopDate)
        
        ## Map visualization parameters
        self.collection = self.collection.map(self.vis_param)
      
        ## Get bounding box
        zVals = np.arange(minZoom,maxZoom+1)
        for z in zVals:
            xVals = self.TileCoordinates(z)
            yVals = self.TileCoordinates(z)
            for x in xVals:
                for y in yVals:
                    # Get bounding box
                    x_TMS, y_TMS = GM.GoogleTile(x,y,z)
                    bbox = list(GM.TileLonLatBounds(x_TMS,y_TMS,z))
            
                    # Replace 180.0 with 179.99
                    bbox = [x if x != 180.0 else 179.99 for x in bbox]
                    bbox = [x if x != -180.0 else -179.99 for x in bbox]
                    
                    ## Area of Interest
                    self.geom = ee.Geometry.Rectangle(bbox)
                    self.region = self.geom.bounds().getInfo()['coordinates'] 
        
                    ## Filter 
                    self.filtered = self.collection.filterBounds(self.geom)
                    
                    ## Export movie-tiles to Google Cloud Storage
                    print('Exporting movie-tile to {}'.format(self.bucket+'/'+self.path+'/'+str(z)+'/'+str(x)+'/'+str(y)+'.mp4'))
                    ee.batch.Export.video.toCloudStorage(
                        collection = self.filtered,
                        description = f'{z}{x}{y}',
                        bucket= self.bucket,
                        fileNamePrefix = self.path+'/'+str(z)+'/'+str(x)+'/'+str(y),
                        dimensions = 256,
                        framesPerSecond = 2,
                        region = self.region).start()

### MODIS Nadir BRDF-Adjusted Reflectance, daily 500m ([gee](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD43A4))
- **Dataset Availability:**
    2000-02-18T00:00:00 - Present
    
As an example we export movi-tiles from a MODIS ImageCollection.

In [3]:
import ee
ee.Initialize()

# Image Collection
imageCollection = 'MODIS/006/MCD43A4'
# Start and stop of time series
startDate = ee.Date('2017-09-01')
stopDate  = ee.Date('2017-10-01')
# The zoom levels of the map tiles to export.
minZoom = 0 
maxZoom = 2
# Image visualization parameters.
visParam = {'bands':['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4','Nadir_Reflectance_Band3'],'min':0,'max':4000.0, 'gamma': 1.4} 
# Google Cloud Bucket
bucket_name = 'skydipper_materials'     
# Folder path in the bucket
folder_path = 'movie-tiles/MODIS'

In [4]:
modis = ee_movie_tiles(imageCollection, startDate, stopDate, minZoom, maxZoom, visParam, bucket_name, folder_path)
modis.export_movie()

Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/0/0/0.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/1/0/0.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/1/0/1.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/1/1/0.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/1/1/1.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/0/0.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/0/1.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/0/2.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/0/3.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/1/0.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/1/1.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/1/2.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/1/3.mp4
Exporting movie-tile to skydipper_materials/movie-tiles/MODIS/2/

## Renaming the `.mp4` files

When exporting the movie-tiles to a Cloud Storage bucket Earth Engine adds a suffix (`ee-export-video` + task_number) at the end of each file name.

To rename the existing files in one of our Cloud Storage buckets:

In [5]:
from google.cloud import storage

In [6]:
def rename_blob(bucket_name, folder_path, privatekey_path):
    """Renames a blob."""
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)

    # List of files
    blobs = bucket.list_blobs(prefix='movie-tiles')

    # Rename '.mp4' files
    for blob in blobs:
        blob_name = blob.name
    
        if ("ee-export-video" in blob_name) and (blob_name[-4:] == '.mp4'):

            new_name = blob_name.split("ee-export-video")[0]+'.mp4'
        
            blob = bucket.blob(blob_name)
            new_blob = bucket.rename_blob(blob, new_name)
        
            print('Blob {} has been renamed to {}'.format(
            blob_name, new_name))

In [7]:
bucket_name = 'skydipper_materials'
folder_path = 'movie-tiles/MODIS'
privatekey_path =  "/Users/keys/privatekey.json"

rename_blob(bucket_name, folder_path, privatekey_path)

Blob movie-tiles/MODIS/1/0/0ee-export-video5923503551241692397.mp4 has been renamed to movie-tiles/MODIS/1/0/0.mp4
Blob movie-tiles/MODIS/1/0/1ee-export-video5257982942738470626.mp4 has been renamed to movie-tiles/MODIS/1/0/1.mp4
Blob movie-tiles/MODIS/1/1/0ee-export-video2710149886668104937.mp4 has been renamed to movie-tiles/MODIS/1/1/0.mp4
Blob movie-tiles/MODIS/2/0/0ee-export-video1469477798348441596.mp4 has been renamed to movie-tiles/MODIS/2/0/0.mp4
Blob movie-tiles/MODIS/2/0/1ee-export-video6312545042210566391.mp4 has been renamed to movie-tiles/MODIS/2/0/1.mp4
Blob movie-tiles/MODIS/2/0/2ee-export-video310073161352993498.mp4 has been renamed to movie-tiles/MODIS/2/0/2.mp4
Blob movie-tiles/MODIS/2/0/3ee-export-video7841597083449843129.mp4 has been renamed to movie-tiles/MODIS/2/0/3.mp4
Blob movie-tiles/MODIS/2/1/0ee-export-video4376602904550588578.mp4 has been renamed to movie-tiles/MODIS/2/1/0.mp4
Blob movie-tiles/MODIS/2/1/1ee-export-video4091285538011975658.mp4 has been renam